In [1]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords # remove all the stop words
from nltk.stem.porter import PorterStemmer # remove the prefix and the suffix, then return the root word
from sklearn.feature_extraction.text import TfidfVectorizer # convert text into feature vectors
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tmakh\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [ ]:
stopwords.words("english")
# these ae the stop words in english

['a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 "he'd",
 "he'll",
 'her',
 'here',
 'hers',
 'herself',
 "he's",
 'him',
 'himself',
 'his',
 'how',
 'i',
 "i'd",
 'if',
 "i'll",
 "i'm",
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it'd",
 "it'll",
 "it's",
 'its',
 'itself',
 "i've",
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'on

Data pre-processing

In [4]:
news_dataset = pd.read_csv("./data/train.csv")

In [5]:
news_dataset.shape

(20800, 5)

In [6]:
news_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [7]:
news_dataset.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [8]:
# we have enough data in this dataset so we want to replace the null values with empty string
# we can do imputation but we have enough data

In [9]:
news_dataset = news_dataset.fillna("")
# replace the missing value with null string

In [10]:
# merging the author column and the news title 
news_dataset["content"] = news_dataset["author"] + " " + news_dataset["title"]

In [12]:
news_dataset["content"]

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object

In [13]:
X = news_dataset.drop(columns="label", axis=1)
y = news_dataset["label"]

In [14]:
X


,id,title,author,text,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,Howard Portnoy Iranian woman jailed for fictio...
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,"Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma..."
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,Michael J. de la Merced and Rachel Abrams Macy...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...","Alex Ansary NATO, Russia To Hold Parallel Exer..."


In [15]:
y

0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 20800, dtype: int64

Stemming

In [16]:
# stem the words: to return the root word
# actor, actress, acting -> act

In [17]:
port_stem = PorterStemmer()

In [20]:
def stemming(content):
    stemmed_content = re.sub("[^a-zA-Z]",' ', content) 
    # if not a to z or A to Z(if not a alphabet) remove it, (remove everything is not an alphabet)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    # all the words will be splitted and converted to a list
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words(["english"])]
    # we are taking each word, except the stopword and stem it(return the root of the word)
    stemmed_content = ' '.join(stemmed_content) # i am joining all the words (they were in a list)
    return stemmed_content

In [21]:
news_dataset["content"] = news_dataset["content"].apply(stemming)

In [22]:
news_dataset["content"]

0        darrel lucu hou dem aid even see comey letter ...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exerci b...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object

In [ ]:
X = news_dataset["content"].values 
y = news_dataset["label"].values
# .values return the values into an array

In [29]:
X


array(['darrel lucu hou dem aid even see comey letter jason chaffetz tweet',
       'daniel j flynn flynn hillari clinton big woman campu breitbart',
       'consortiumnew com truth might get fire', ...,
       'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time',
       'alex ansari nato russia hold parallel exerci balkan',
       'david swanson keep f aliv'], shape=(20800,), dtype=object)

In [25]:
y

array([1, 0, 1, ..., 0, 1, 1], shape=(20800,))

Converting Textual Data to Numerical Data

In [30]:
vectorizer = TfidfVectorizer()
vectorizer.fit(X) 

X = vectorizer.transform(X)

In [32]:
print(X)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 210584 stored elements and shape (20800, 16984)>
  Coords	Values
  (0, 263)	0.2701012497770876
  (0, 2464)	0.36765196867972083
  (0, 2933)	0.24684501285337127
  (0, 3567)	0.3598939188262558
  (0, 3759)	0.27053324808454915
  (0, 4917)	0.23331696690935097
  (0, 6930)	0.2187416908935914
  (0, 7612)	0.24785219520671598
  (0, 8546)	0.2921251408704368
  (0, 8822)	0.36359638063260746
  (0, 13348)	0.2565896679337956
  (0, 15553)	0.2848506356272864
  (1, 1481)	0.2957471154505952
  (1, 1877)	0.15614790568229528
  (1, 2206)	0.36915639258038363
  (1, 2790)	0.19208753385709676
  (1, 3535)	0.2653147533915268
  (1, 5440)	0.7186013955384664
  (1, 6744)	0.19152496072048605
  (1, 16656)	0.3025156488372128
  (2, 2917)	0.3179886800654691
  (2, 3072)	0.46097489583229645
  (2, 5326)	0.3866530551182615
  (2, 5903)	0.3474613386728292
  (2, 9532)	0.49351492943649944
  :	:
  (20797, 3610)	0.211655450844435
  (20797, 6969)	0.21809398920480086
  (20797

In [33]:
# Splitting the dataset into training and testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state= 2)

In [34]:
# Training the model: Logistic Regression

In [35]:
model = LogisticRegression()

In [36]:
model.fit(X_train, y_train)

LogisticRegression()

Evaluation

In [37]:
# accuracy socre on the training data 
X_train_prediction = model.predict(X_train) # make our model to predict in the training data, return the labels predicted
training_data_accuracy = accuracy_score(X_train_prediction, y_train)

In [38]:
training_data_accuracy

0.9863581730769231

In [ ]:
# accuracy socre on the training data 
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, y_test)

In [40]:
test_data_accuracy

0.9790865384615385

Build a predictive system

In [43]:
X_new = X_test[0] # the first row(news) in the test data

prediction = model.predict(X_new)
print(prediction)

if (prediction[0] == 0):
    print("The news is Real")
else:
    print("The news is Fake")

[1]
The news is Fake


In [ ]:
print(y_test[0])
# 1 so the prediction is correct for the first row

1
